In [603]:
import datetime
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from PyPDF2 import PdfReader, PdfWriter
from reportlab.lib.utils import ImageReader  # Import ImageReader for BytesIO compatibility
from reportlab.lib.colors import Color
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import seaborn as sns
%matplotlib inline
# Replace 'file_path.csv' with the path to your CSV file
df = pd.read_csv('input.csv')

# Display the first few rows of the DataFrame
print(df.head())

     Student ID   Name of the student             School Name  Class District  \
0  23JNGZPDH001          P. Vaishnavi  ZPHS Boys Dharmakancha      9  Jangaon   
1  23JNGZPDH002            B. Sanjana  ZPHS Boys Dharmakancha      9  Jangaon   
2  23JNGZPDH003          K. Elizabeth  ZPHS Boys Dharmakancha      9  Jangaon   
3  23JNGZPDH004             P. Sarika  ZPHS Boys Dharmakancha      9  Jangaon   
4  23JNGZPDH005  N. Vaishnavi Sridevi  ZPHS Boys Dharmakancha      9  Jangaon   

   Email  Google Tools  Google for Information & Internet Ethics and Security  \
0     90            73                                                 80       
1     80            64                                                 27       
2     20            58                                                 60       
3     20            32                                                 27       
4     20            63                                                 73       

   Google Docs & Sheets  F

In [604]:
# Register fonts
pdfmetrics.registerFont(TTFont('Playball-Regular', './fonts/Playball/Playball-Regular.ttf'))
pdfmetrics.registerFont(TTFont('Arimo-Regular', './fonts/Arimo/static/Arimo-Regular.ttf'))
pdfmetrics.registerFont(TTFont('Arimo-Bold', './fonts/Arimo/static/Arimo-Bold.ttf'))

# Center position of page
page_width, page_height = A4
center_x = page_width / 2

def draw_centered_text(canvas, segments, y_position):
    """Helper function to draw text centered at the specified y-position."""
    total_width = sum(pdfmetrics.stringWidth(text, font, size) for text, font, size in segments)
    start_x = center_x - (total_width / 2)
    for text, font, size in segments:
        canvas.setFont(font, size)
        canvas.drawString(start_x, y_position, text)
        start_x += pdfmetrics.stringWidth(text, font, size)

def generate_student_graph(data):
    """Generates a high-definition bar chart for a student's scores as an in-memory image without a border."""
    # Prepare data
    count_df = pd.Series(data).to_frame('Score')
    count_df.index.name = 'Category'
    count_df.reset_index(drop=False, inplace=True)
    
    # Set up plot style
    sns.set(style="white", rc={"lines.linewidth": 3})
    fig, ax = plt.subplots(figsize=(11, 3.5))  # Adjusted size to fit blue box width

    # Plot the bar chart
    sns.barplot(x='Score', y='Category', data=count_df, color='#3592f0', ax=ax)
    
    # Adjust x-axis limit and grid for consistency
    ax.set_xlim(0, 100)  # Set x-axis limit if scores are out of 100
    ax.xaxis.grid(color='gray', linestyle='dashed')
    
    # Remove default spines
    sns.despine(left=False, bottom=False)  # Keep left and bottom spines
    ax.set_ylabel('')  # Remove "Category" label from y-axis
    
    # Remove extra padding around the plot
    plt.subplots_adjust(left=0.01, right=0.99, top=0.99, bottom=0.01)  # Minimizing the padding
    
    # Save plot to a BytesIO buffer with high DPI and transparent background
    buffer = BytesIO()
    plt.savefig(buffer, format='PNG', dpi=300, bbox_inches='tight', transparent=True)  # Transparent background
    plt.close(fig)
    buffer.seek(0)  # Rewind buffer to the beginning
    
    # Convert to ImageReader for ReportLab compatibility
    return ImageReader(buffer)

def create_overlay(student_data):
    packet = BytesIO()
    c = canvas.Canvas(packet, pagesize=A4)

    # Center position for Student Name with blue color
    c.setFont("Playball-Regular", 42.66)
    c.setFillColorRGB(0.29, 0.53, 0.77)  # Blue color for Student Name
    student_name = student_data['Name of the student']
    text_width = pdfmetrics.stringWidth(student_name, "Playball-Regular", 42.66)
    start_x = center_x - (text_width / 2)
    c.drawString(start_x, 627.0, student_name)

    # Reset color to black for the rest of the text
    c.setFillColorRGB(0, 0, 0)  # Black color

     # Line 1: "of <School’s Name>, <District> has successfully completed"
    y_position = 590.35
    line1_segments = [
        ("of ", "Arimo-Regular", 14.58),
        (student_data['School Name'], "Arimo-Bold", 14.58),
        (", ", "Arimo-Regular", 14.58),
        (student_data['District'], "Arimo-Bold", 14.58),
        (" has successfully completed the ", "Arimo-Regular", 14.58)
    ]
    draw_centered_text(c, line1_segments, y_position)

    # Line 2: "Digital Essentials Program conducted by facilitators of Digital Equity"
    y_position = 572.35
    line2_segments = [
        ("Digital Essentials", "Arimo-Bold", 14.58),
        (" Program conducted by facilitators of ", "Arimo-Regular", 14.58),
        ("Digital Equity", "Arimo-Bold", 14.58)
    ]
    draw_centered_text(c, line2_segments, y_position)

    # Line 3: "for the year <Current Year>"
    y_position = 553
    current_year = str(datetime.datetime.now().year)
    line3_segments = [
        ("for the year ", "Arimo-Regular", 14.58),
        (current_year, "Arimo-Bold", 14.58)
    ]
    draw_centered_text(c, line3_segments, y_position)

    # Inject Total Score and Proficiency Level values at specified positions
    c.setFont("Arimo-Bold", 14.58)
    c.drawString(300, 503, f"{student_data['Total']}")  # Position for Total Score
    c.drawString(300, 483, student_data['Proficiency Level'])  # Position for Proficiency Level

    # Footer: Center-align School Name and District under "Principal"
    footer_y_position = 22  # Adjust based on where you want it relative to "Principal"
    school_name = student_data['School Name']
    district = student_data['District']

    # Center-align School Name
    school_name_width = pdfmetrics.stringWidth(school_name, "Arimo-Regular", 12.51)
    school_name_x = 515 - (school_name_width / 2)  # Adjust x-position relative to "Principal" position
    c.setFont("Arimo-Regular", 12.51)
    c.drawString(school_name_x, footer_y_position, school_name)

    # Center-align District beneath the School Name
    district_width = pdfmetrics.stringWidth(district, "Arimo-Regular", 12.51)
    district_x = 515 - (district_width / 2)  # Adjust x-position relative to "Principal" position
    c.drawString(district_x, footer_y_position - 15, district)  # Positioned slightly lower than school name

    # Draw the rounded rectangle for the graph background
    border_color = Color(0.29, 0.53, 0.77)  # Same color as student name
    c.setStrokeColor(border_color)
    c.setLineWidth(2)
    c.roundRect(75, 150, 500, 250, 12, stroke=True, fill=False)

    # Generate and insert graph without border
    graph_data = {
        "Email": student_data['Email'],
        "Google Tools": student_data['Google Tools'],
        "Google for Information & Internet Ethics and Security": student_data['Google for Information & Internet Ethics and Security'],
        "Google Docs & Sheets": student_data['Google Docs & Sheets'],
        "Final Project": student_data['Final Project'],
        "Facilitator student assessment": student_data['Facilitator student assessment'],
        "Research, Documentation, Data Analysis & Presentation": student_data['Research, Documentation, Data Analysis & Presentation']
    }
    graph_image = generate_student_graph(graph_data)

    # Insert the graph image in the blue box
    c.drawImage(graph_image, x=75, y=180, width=490, height=240)  # Adjust slightly smaller than the border

    # Finalize the overlay
    c.save()
    packet.seek(0)
    return PdfReader(packet)

def create_certificate(student_data, template_path, output_file):
    # Read the template PDF
    template_pdf = PdfReader(template_path)
    template_page = template_pdf.pages[0]

    # Create an overlay with student data
    overlay_pdf = create_overlay(student_data)
    overlay_page = overlay_pdf.pages[0]

    # Merge the template and overlay
    template_page.merge_page(overlay_page)

    # Write the merged content to a new PDF
    pdf_writer = PdfWriter()
    pdf_writer.add_page(template_page)

    with open(output_file, "wb") as out_pdf:
        pdf_writer.write(out_pdf)

In [605]:
# Generate a certificate for each student
for index, row in df.iterrows():
    if index < 3:
        student_id = row['Student ID'].replace(" ", "_")
        output_file = f"./out/{student_id}_certificate.pdf"
        create_certificate(row, 'template_blank.pdf',output_file)
        print(f"Certificate generated for {row['Name of the student']} - saved as {output_file}")

Certificate generated for P. Vaishnavi - saved as ./out/23JNGZPDH001_certificate.pdf
Certificate generated for B. Sanjana - saved as ./out/23JNGZPDH002_certificate.pdf
Certificate generated for K. Elizabeth - saved as ./out/23JNGZPDH003_certificate.pdf
